In [ ]:
import numpy as np
import flammkuchen as fl
import napari
from pathlib import Path

from split_dataset import SplitDataset
import json

import flammkuchen as fl 
import tifffile as tiff

In [ ]:
master_path = Path(r"\\portulab.synology.me\data\Hagar and Ot\E0040\v10\LS ablation\ntr")
fish_list = list(master_path.glob("*_f*"))

unexported = [f for f in fish_list if not (f / "data_from_suite2p_cells_brain.h5").exists()]
path = unexported[0]

In [ ]:
suite2p_data = fl.load(path / "data_from_suite2p_cells.h5")
traces = suite2p_data['traces']
roi_stack = suite2p_data['rois_stack']
all_coords = suite2p_data['coords']

In [ ]:
anatomy = suite2p_data['anatomy_stack']
tiff.imsave(path / "anatomy_suite2p.tif", anatomy)

In [ ]:
viewer = napari.view_image(anatomy,  colormap="gray")
brain_area = np.zeros(np.shape(anatomy), dtype=int)
labels_layer = viewer.add_labels(brain_area, name='brain_area')

In [ ]:
#### Run the next cells only after marking the regions you want to keep in napari

In [ ]:
labels = viewer.layers["brain_area"].data

In [ ]:
coords_to_remove = []
coords_to_keep = []
num_cells = np.shape(all_coords)[0]
for i in range(num_cells):
    tmp_cell = labels[int(all_coords[i, 0]), int(all_coords[i, 1]), int(all_coords[i, 2])]
    if tmp_cell < 1:
        coords_to_remove = coords_to_remove + [i]
    else:
        coords_to_keep = coords_to_keep + [i]

In [ ]:
print(len(coords_to_remove))
print(len(coords_to_keep))

In [ ]:
traces_ib = traces[np.asarray(coords_to_keep)]
coords_ib = all_coords[np.asarray(coords_to_keep)]

In [ ]:
d = {
    'traces': traces_ib,
    'coords': coords_ib,
    'coords_idx': coords_to_keep,
    'coords_removed': coords_to_remove,
}
fl.save(path / 'data_from_suite2p_cells_brain.h5', d)
